# Functioneer Examples

This notebook tests the core functionality of the `functioneer` library, ensuring all existing and new features work as expected. We use the Rosenbrock function as our Example due to its simplicity, multiple inputs, and historical significance.

## Examples
1. Basic Parameter Definition and Function Execution
2. Single Parameter Forks
3. Optimization
4. Multi-parameter Forks
5. Conditional Analysis Steps
6. Dictionary Output in Execute (for when multiple return values are needed)
7. Optimizer Options
8. Custom Optimzer

In [1]:
# Setup: Import libraries and define the Rosenbrock function
import functioneer as fn
from scipy.optimize import minimize

# Rosenbrock function (known minimum of 0 at: x=1, y=1, a=1, b=100)
def rosenbrock(x, y, a, b):
    return (a - x)**2 + b * (y - x**2)**2

In [ ]:
# Example 1: Basic Parameter Definition and Function Execution
anal = fn.AnalysisModule()  # its not ānal is anál!
anal.add.define('a', 1.0) # Define parameter 'a' and set value to 1.0
anal.add.define('b', 100)
anal.add.define('x', 1)
anal.add.define('y', 1)
anal.add.fork()
anal.add.execute(func=rosenbrock) # Execute function with parameter ids matched to kwargs
results = anal.run()
print('Example 1 Output:')
print(results['df'])

Example 1 Output:
   runtime    a    b  x  y  rosenbrock                   datetime
0      0.0  1.0  100  1  1         0.0 2025-06-25 14:46:27.216448


In [3]:
# Example 2: Single Parameter Forks
init_params = dict(a=1, b=100, x=1, y=1)
anal = fn.AnalysisModule(init_params)
anal.add.fork('x', (0, 1, 2)) # Fork analysis, create a branch for each value of 'x': 0, 1, 2
anal.add.fork('y', (1, 10))
anal.add.execute(func=rosenbrock, assign_to='brock_purdy') # set custom param name with assign_to
results = anal.run()
print('\nExample 2 Output:')
print(results['df'].drop(columns='datetime')) # drop datetime for shorter prints


Example 2 Output:
    runtime  a    b  x   y  brock_purdy
0  0.001001  1  100  0   1          101
1  0.000000  1  100  0  10        10001
2  0.000000  1  100  1   1            0
3  0.000000  1  100  1  10         8100
4  0.000000  1  100  2   1          901
5  0.000000  1  100  2  10         3601


In [4]:
# Example 3: Optimization
anal = fn.AnalysisModule(dict(x=0, y=0))
anal.add.fork('a', value_set=(1, 2))
anal.add.fork('b', value_set=(0, 100, 200))
anal.add.optimize(func=rosenbrock, opt_param_ids=('x', 'y')) # Optimize 'x' and 'y' to minimize rosenbrock
results = anal.run()
print('\nExample 3 Output:')
print(results['df'].drop(columns='datetime'))


Example 3 Output:
    runtime         x         y  a    b    rosenbrock
0  0.002999  1.000000  0.000000  1    0  4.930381e-32
1  0.014998  0.999763  0.999523  1  100  5.772481e-08
2  0.009740  0.999939  0.999873  1  200  8.146869e-09
3  0.001345  2.000000  0.000000  2    0  0.000000e+00
4  0.006150  1.999731  3.998866  2  100  4.067518e-07
5  0.018234  1.999554  3.998225  2  200  2.136755e-07


In [5]:
# Example 4: Multi-parameter Fork and Define
anal = fn.AnalysisModule()
anal.add.define({'a': 1, 'b': 100})
anal.add.fork({'x': (0, 1, 2), 'y': (0, 10, 20)})
anal.add.execute(func=rosenbrock)
results = anal.run()
print('\nExample 4 Output:')
print(results['df'].drop(columns='datetime'))


Example 4 Output:
   runtime  a    b  x   y  rosenbrock
0      0.0  1  100  0   0           1
1      0.0  1  100  1  10        8100
2      0.0  1  100  2  20       25601


In [6]:
# Example 5: Conditional Analysis Steps
expensive_func = lambda x, y: x + y

anal = fn.AnalysisModule(dict(x=0, y=0))
anal.add.fork('a', value_set=(1, 2))
anal.add.fork('b', value_set=(0, 100, 200))
anal.add.optimize(func=rosenbrock, opt_param_ids=('x', 'y'))

# Only evaluate 'expensive_func' if the optimized 'y' is above 0.5
anal.add.execute(func=expensive_func, assign_to='expensive_param', condition=lambda y: y > 0.5)
results = anal.run()
print('\nExample 5 Output:')
print(results['df'].drop(columns='datetime'))


Example 5 Output:
    runtime         x         y  a    b    rosenbrock  expensive_param
0  0.001993  1.000000  0.000000  1    0  4.930381e-32              NaN
1  0.008992  0.999763  0.999523  1  100  5.772481e-08         1.999286
2  0.007304  0.999939  0.999873  1  200  8.146869e-09         1.999811
3  0.000993  2.000000  0.000000  2    0  0.000000e+00              NaN
4  0.009903  1.999731  3.998866  2  100  4.067518e-07         5.998596
5  0.007782  1.999554  3.998225  2  200  2.136755e-07         5.997779


In [7]:
# Example 6: Dictionary Output in Execute
# useful for functions that need to return multiple values
def rosenbrock_dict(x, y, a, b):
    val = (a - x)**2 + b * (y - x**2)**2
    return {'rosen': val, 'sum': x + y}

anal = fn.AnalysisModule(dict(a=1, b=100, x=1, y=1))
anal.add.execute(func=rosenbrock_dict, assign_to=['rosen', 'sum'], unpack_result=True)
results = anal.run()
print('\nExample 6 Output:')
print(results['df'].drop(columns='datetime'))


Example 6 Output:
   runtime  a    b  x  y  rosen  sum
0      0.0  1  100  1  1      0    2


In [8]:
# Example 7: Optimizer Options
def rosenbrock_neg(x, y, a, b):
    val = (a - x)**2 + b * (y - x**2)**2
    return -val

anal = fn.AnalysisModule(dict(a=1, b=100, x=5, y=4))
anal.add.optimize(
    func=rosenbrock_neg, 
    opt_param_ids=('x', 'y'), 
    direction='max', # default is 'min'
    optimizer='Nelder-Mead', 
    bounds=dict(x=(-100,100), y=(-50, 50)),
    tol=1e-2
)

results = anal.run()
print('\nExample 7 Output:')
print(results['df'].drop(columns='datetime'))


Example 7 Output:
   runtime  a    b         x         y  rosenbrock_neg
0  0.01311  1  100  1.001503  1.002813       -0.000006


In [9]:
# Example 8: Custom Optimizer
anal = fn.AnalysisModule(dict(a=1, b=100, x=5, y=4))

def custom_optimizer(func, x0, **kwargs):
    return minimize(func, x0, method='BFGS', options={'gtol': 1e-6})
anal.add.optimize(func=rosenbrock, opt_param_ids=['x', 'y'], optimizer=custom_optimizer)

results = anal.run()
print('\nExample 8 Output:')
print(results['df'].drop(columns='datetime'))


Example 8 Output:
    runtime  a    b         x         y    rosenbrock
0  0.030716  1  100  0.999996  0.999991  2.003596e-11
